In [47]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
df=pd.read_json("/Users/David/code/Clementinelalande/marbotic/raw_data/export_23-5_minified.json")

In [79]:
df.shape

(1320229, 16)

In [80]:
df.columns

Index(['city', 'client_event_time', 'country', 'device_model', 'event_id',
       'event_type', 'is_attribution_event', 'language', 'os_version',
       'region', 'session_id', 'start_version', 'user_creation_time',
       'user_id', 'user_properties', 'event_properties'],
      dtype='object')

In [81]:
df.event_type.unique()

array(['[Activity] Good Answer', '[Activity] Wrong Answer',
       '[Activity] Stop', '[Activity] Start', '[Action] Click',
       '[Scene] Leave', '[Scene] Enter', '[Game] Pause',
       '[Scaffolding] Scaffolding', '[Game] Start', '[Error] Error',
       '[Game] Stop', '[OB] CommPreview', '[OB] DoYouHaveKit',
       '[OB] HasKitYes', '[OB] Email', '[OB] VerifCode',
       '[Activation] Game', '[OB] HasKitNo', '[OB] DiscoverTrial',
       '[OB] ClickTrial', '[Account] Sign In', '[Profile] Create',
       '[Profile] Update', '[OB] ProductSelection', '[OB] ProductHelp',
       '[OB] Type', '[Account] Sign Up', '[OB] ProfileName',
       '[OB] ProfileBirthdate', '[Activation] New Activation',
       '[Activation] Product', '[Activation] Piece', '[OB] Success',
       '[OB] ProductActivation', '[Scaffolding] Reset', '[OB] TrialStart',
       '[Account] Update Request', '[Profile] Delete', '[Popup] Open',
       '[Popup] Close', '[Account] Update Success', '[Account] Sign Out',
       '[Pr

In [82]:
df.head()

,city,client_event_time,country,device_model,event_id,event_type,is_attribution_event,language,os_version,region,session_id,start_version,user_creation_time,user_id,user_properties,event_properties
0,Dallas,2022-03-01 00:57:25.903,United States,"iPad13,4",1110,[Activity] Good Answer,False,English,15.3,Texas,1646095346396,1.0.7,2021-12-28 01:29:38.094,67531,"{'Newsletter': 'False', 'Products': ['Smart Le...","{'[Activity] Name': 'LinePanties', '[Activity]..."
1,Dallas,2022-03-01 00:57:41.878,United States,"iPad13,4",1111,[Activity] Wrong Answer,False,English,15.3,Texas,1646095346396,1.0.7,2021-12-28 01:29:38.094,67531,"{'Newsletter': 'False', 'Products': ['Smart Le...","{'[Activity] Name': 'LinePanties', '[Activity]..."
2,Toronto,2022-03-01 00:57:28.175,Canada,"iPad12,1",1430,[Activity] Stop,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882,"{'Newsletter': 'True', 'Products': ['Smart Let...","{'[Activity] Name': '1.2', '[Game] Code': 'Mar..."
3,Toronto,2022-03-01 00:57:28.177,Canada,"iPad12,1",1431,[Activity] Start,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882,"{'Newsletter': 'True', 'Products': ['Smart Let...","{'[Activity] Name': '1.3', '[Game] Code': 'Mar..."
4,Toronto,2022-03-01 00:57:38.278,Canada,"iPad12,1",1432,[Activity] Stop,False,English,15.2,Ontario,1646096085235,1.3.1,2021-07-19 20:24:36.326,60882,"{'Newsletter': 'True', 'Products': ['Smart Let...","{'[Activity] Name': '1.3', '[Game] Code': 'Mar..."


In [109]:
df.event_properties[4]

{'[Activity] Name': '1.3',
 '[Game] Code': 'Marbotic',
 '[Time] Spent': 10,
 '[Scene] Name': 'Counting_Home',
 '[Profile] Age (days)': 2423,
 '[Game] Language': 'en_US',
 '[Activity] Solved': False,
 '[Scaffolding] Level': 11,
 '[Scene] Section': 'UP100'}

In [130]:
df.user_properties[4]

{'Newsletter': 'True',
 'Products': ['Smart Letters', 'Smart Numbers'],
 'Pieces': ['Uppercase Letters', 'Numbers'],
 'Language': 'English',
 'Type': 'Parent',
 'Country': 'CA',
 'Games': ['Marbotic Numbers',
  'Marbotic Letters',
  'Alphamonster',
  'Vocabubble',
  'Bla Bla Box',
  'Lil Reader',
  'Up To 100',
  '10 Fingers',
  'More Or Less',
  'Marbotic']}

In [83]:
user_properties_df=df.copy()

In [84]:
user_properties=user_properties_df[["user_id","user_properties"]]

In [85]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=PoQsXFrUUsLui0bniQSz9njix6vkmV&access_type=offline&code_challenge=S3iaAqXEe7U5N46YKdeJCKShEw-o-S2tI7TMNTfuXWM&code_challenge_method=S256


You are now logged in as [david.periers@gmail.com].
Your current project is [marbotic].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [86]:
!gcloud projects list

PROJECT_ID              NAME              PROJECT_NUMBER
marbotic                Marbotic          566474310042
symmetric-curve-346916  My First Project  175516577791
wagon-bootcamp-346916   Wagon bootcamp    697736453840


In [87]:
!gcloud config set project marbotic

Updated property [core/project].


In [88]:
!bq ls

/usr/local/Caskroom/google-cloud-sdk/latest/google-cloud-sdk/platform/bq/bq.py:41: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
     datasetId      
 ------------------ 
  marbotic_dataset  


In [89]:
!cat credentials

cat: credentials: No such file or directory


In [93]:
from google.oauth2 import service_account

key_path = "/Users/David/code/dperiers/GITHUB_NICKNAME/gcp/marbotic-587600c2c13f.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [131]:
from google.oauth2 import service_account

def extract():
    # Construct a BigQuery client object.

    key_path = "/Users/David/code/dperiers/GITHUB_NICKNAME/gcp/marbotic-587600c2c13f.json"

    credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],)
    
    client = bigquery.Client(project="marbotic", credentials=credentials)

    query = """
        SELECT *
        FROM `marbotic.marbotic_dataset.events`
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [132]:
extract()

,user_id,start_version,os_version,client_event_time,language,event_id,session_id,region,is_attribution_event,user_creation_time,device_model,event_type,country,city
0,69077.0,1.0.8,10.3.3,2022-03-01 18:11:29.219000+00:00,English,7,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[Account] Sign In,Canada,North Vancouver
1,69077.0,1.0.8,10.3.3,2022-03-01 18:20:11.873000+00:00,English,29,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[OB] ProductSelection,Canada,North Vancouver
2,69077.0,1.0.8,10.3.3,2022-03-01 18:20:19.948000+00:00,English,30,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[OB] ProductHelp,Canada,North Vancouver
3,69077.0,1.0.8,10.3.3,2022-03-01 18:20:22.647000+00:00,English,31,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[OB] ProductActivation,Canada,North Vancouver
4,69077.0,1.0.8,10.3.3,2022-03-01 18:21:22.139000+00:00,English,33,1646158215337,British Columbia,False,2022-01-24 05:21:26.336000+00:00,iPad 4,[Scaffolding] Reset,Canada,North Vancouver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,68169.0,1.0.7,14.6,2022-03-13 10:17:08.262000+00:00,French,1308,1647165206796,Maine-et-Loire,False,2022-01-08 09:38:37.611000+00:00,iPad 5,[Scaffolding] Scaffolding,France,Gennes
399996,68169.0,1.0.7,14.6,2022-03-13 10:27:33.469000+00:00,French,1357,1647165206796,Maine-et-Loire,False,2022-01-08 09:38:37.611000+00:00,iPad 5,[Scaffolding] Scaffolding,France,Gennes
399997,68169.0,1.0.7,14.6,2022-03-13 10:23:30.013000+00:00,French,1343,1647165206796,Maine-et-Loire,False,2022-01-08 09:38:37.611000+00:00,iPad 5,[Scaffolding] Scaffolding,France,Gennes
399998,68169.0,1.0.7,14.6,2022-03-13 10:03:57.818000+00:00,French,1247,1647165206796,Maine-et-Loire,False,2022-01-08 09:38:37.611000+00:00,iPad 5,[Scaffolding] Scaffolding,France,Gennes


In [ ]:
test=pd.DataFrame(columns=["a","b"])
test

In [104]:
data=pd.read_csv("/Users/David/code/Clementinelalande/marbotic/raw_data/users_info_202206071551.csv",sep=";")

In [105]:
data.head()

,User id,Country,Language,News letter,Type,Attribution,Games,Products,Pieces,Purchases
0,59707,MX,English,True,Parent,PLU,"[""10 Fingers"",""Marbotic Letters"",""Marbotic Num...","[""Smart Kit"",""Smart Numbers"",""Nordic Kit""]","[""Nordic Letters"",""Numbers"",""Uppercase Letters""]",NaN
1,61748,NZ,English,False,Educator,ELU,"[""Marbotic Letters"",""Vocabubble"",""10 Fingers"",...","[""Smart Letters"",""Smart Numbers""]","[""Numbers"",""Uppercase Letters""]",NaN
2,54834,CA,English,True,Parent,PLU,"[""Sesame Street Numbers""]",NaN,NaN,NaN
3,69202,US,English,False,Parent,PLU,"[""Marbotic""]","[""Deluxe Learning Kit""]","[""Lowercase Letters"",""Numbers""]",NaN
4,67754,CA,English,True,Parent,PLU,"[""Marbotic""]","[""Smart Kit""]","[""Numbers"",""Uppercase Letters""]",NaN


In [111]:
data.shape

(13679, 10)

In [129]:
len(data["User id"].unique())

12858